# Retail Sales Cleaning & KPI Dashboard Prep

**Goal:** Build a reproducible, portfolio-ready workflow that cleans a messy retail sales file, documents all fixes, computes key KPIs, and prepares Excel-ready tables for a sales dashboard.

**Dataset:** Kaggle Online Retail (messy version stored as `data/raw/messy_raw.csv`).

In this notebook we will:

- Inspect the messy input file and highlight data quality issues.
- Apply explicit, documented cleaning rules to create a standardized `cleaned_sales.csv`.
- Produce a `validation_summary.csv` that quantifies what was removed or fixed.
- Compute a small set of core sales KPIs and export chart-ready tables.
- Generate a data dictionary and an Excel-ready package for building a dashboard.

All steps are designed to run end-to-end starting from `messy_raw.csv` with no manual edits in between.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

RAW = PROJECT_ROOT / "data" / "raw"
PROCESSED = PROJECT_ROOT / "data" / "processed"
OUTPUTS = PROJECT_ROOT / "outputs"
SCREENSHOTS = PROJECT_ROOT / "screenshots"

for folder in [PROCESSED, OUTPUTS, SCREENSHOTS]:
    folder.mkdir(parents=True, exist_ok=True)

RAW, PROCESSED, OUTPUTS

(PosixPath('/content/data/raw'),
 PosixPath('/content/data/processed'),
 PosixPath('/content/outputs'))

In [2]:
DATE_COL = "InvoiceDate"
QTY_COL = "Quantity"
PRICE_COL = "UnitPrice"
ORDER_COL = "InvoiceNo"
PRODUCT_COL = "Description"
CUSTOMER_COL = "CustomerID"
REGION_COL = "Country"

CONFIG_COLS = {
    "DATE_COL": DATE_COL,
    "QTY_COL": QTY_COL,
    "PRICE_COL": PRICE_COL,
    "ORDER_COL": ORDER_COL,
    "PRODUCT_COL": PRODUCT_COL,
    "CUSTOMER_COL": CUSTOMER_COL,
    "REGION_COL": REGION_COL,
}

CONFIG_COLS

{'DATE_COL': 'InvoiceDate',
 'QTY_COL': 'Quantity',
 'PRICE_COL': 'UnitPrice',
 'ORDER_COL': 'InvoiceNo',
 'PRODUCT_COL': 'Description',
 'CUSTOMER_COL': 'CustomerID',
 'REGION_COL': 'Country'}

In [5]:
raw_path = RAW / "messy_raw.csv"

df_raw = pd.read_csv(raw_path, encoding="ISO-8859-1")

required_cols = [DATE_COL, QTY_COL, PRICE_COL, ORDER_COL, PRODUCT_COL]
optional_cols = [CUSTOMER_COL, REGION_COL]

missing_required = [c for c in required_cols if c not in df_raw.columns]
if missing_required:
    raise ValueError(
        f"Config column(s) not found in data: {missing_required}. "
        f"Please update the CONFIG section to match your dataset's column names."
    )

print("Raw shape:", df_raw.shape)
print("Columns:", df_raw.columns.tolist())
df_raw.head()

Raw shape: (541929, 8)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,NaN,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [6]:
def to_number(series: pd.Series) -> pd.Series:
    return pd.to_numeric(
        series.astype(str).str.replace(",", ".", regex=False),
        errors="coerce"
    )

qty_raw = to_number(df_raw[QTY_COL])
price_raw = to_number(df_raw[PRICE_COL])

missing_product = df_raw[PRODUCT_COL].isna().sum()

missing_customer = (
    df_raw[CUSTOMER_COL].isna().sum()
    if CUSTOMER_COL in df_raw.columns else np.nan
)

missing_region = (
    df_raw[REGION_COL].isna().sum()
    if REGION_COL in df_raw.columns else np.nan
)

neg_qty = (qty_raw < 0).sum()
neg_price = (price_raw < 0).sum()

issues_summary = pd.DataFrame(
    {
        "rows_in_raw": [len(df_raw)],
        f"{PRODUCT_COL}_missing": [missing_product],
        f"{CUSTOMER_COL}_missing": [missing_customer],
        f"{REGION_COL}_missing": [missing_region],
        "negative_quantity": [neg_qty],
        "negative_price": [neg_price],
    }
)

issues_summary

,rows_in_raw,Description_missing,CustomerID_missing,Country_missing,negative_quantity,negative_price
0,541929,7494,138735,6035,10545,2


## Cleaning Rules

The following rules are applied **in code** to transform `messy_raw.csv` into a clean, analysis-ready dataset:

- **Duplicates removal**
  - Remove **exact duplicate rows** using `drop_duplicates()` across all columns.
- **Date parsing**
  - Parse `DATE_COL` into a single standardized `datetime64` column.
  - Use `pd.to_datetime(..., errors="coerce", dayfirst=True)` to handle multiple formats.
  - Rows with unparseable dates (date becomes `NaT`) are counted as **bad dates** and removed.
- **Numeric coercion (with comma decimals)**
  - Convert `QTY_COL` and `PRICE_COL` to numeric after replacing commas with dots.
  - Non-numeric values are coerced to `NaN`.
- **Text normalization**
  - For text fields (`ORDER_COL`, `PRODUCT_COL`, `CUSTOMER_COL`, `REGION_COL` where present):
    - Strip leading/trailing whitespace.
    - Normalize repeated internal spaces.
    - Keep original case for readability (no forced upper/lower casing).
- **Invalid row rules**
  - A row is marked **invalid** and removed if any of the following are true:
    - Quantity is missing or non-numeric (`NaN`) or `<= 0`.
    - Unit price is missing or non-numeric (`NaN`) or `<= 0`.
    - Product description is missing.
    - Date could not be parsed to a valid datetime.
    - (If present) Region is missing.
- **Revenue computation**
  - Create a `Revenue` field as `Quantity * UnitPrice` **after** cleaning and numeric coercion.
- **Handling missing customers**
  - Rows with missing customers are **kept** in the cleaned data but counted in the validation summary.
  - Customer ID remains missing to reflect the original data quality.
- **Definition of “unfixable”**
  - A row is considered **unfixable** if it violates any of the core data-quality rules above (bad date, invalid or missing quantity/price, missing product, or missing region where required) and is therefore removed from the cleaned dataset.

In [7]:


df = df_raw.copy()

rows_in = len(df)

text_cols = [ORDER_COL, PRODUCT_COL, CUSTOMER_COL, REGION_COL]
for col in text_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
            .replace({"nan": np.nan})
        )

df["_parsed_date"] = pd.to_datetime(
    df[DATE_COL],
    errors="coerce",
    dayfirst=True,
    infer_datetime_format=True,
)

bad_dates_mask = df["_parsed_date"].isna()
bad_dates_removed = int(bad_dates_mask.sum())

df = df[~bad_dates_mask].copy()
df[DATE_COL] = df["_parsed_date"]
df.drop(columns=["_parsed_date"], inplace=True)

def to_number(series: pd.Series) -> pd.Series:
    return pd.to_numeric(
        series.astype(str).str.replace(",", ".", regex=False),
        errors="coerce"
    )

df[QTY_COL] = to_number(df[QTY_COL])
df[PRICE_COL] = to_number(df[PRICE_COL])

qty_nan = int(df[QTY_COL].isna().sum())
price_nan = int(df[PRICE_COL].isna().sum())

before_dups = len(df)
df = df.drop_duplicates()
duplicates_removed = int(before_dups - len(df))

invalid_mask = (
    df[QTY_COL].isna()
    | df[PRICE_COL].isna()
    | (df[QTY_COL] <= 0)
    | (df[PRICE_COL] <= 0)
    | df[PRODUCT_COL].isna()
)

customer_missing = (
    int(df[CUSTOMER_COL].isna().sum())
    if CUSTOMER_COL in df.columns else np.nan
)

if REGION_COL in df.columns:
    invalid_mask = invalid_mask | df[REGION_COL].isna()

invalid_rows_removed = int(invalid_mask.sum())

df_clean = df[~invalid_mask].copy()

df_clean["Revenue"] = df_clean[QTY_COL] * df_clean[PRICE_COL]

df_clean = df_clean.sort_values([DATE_COL, ORDER_COL]).reset_index(drop=True)

rows_out = len(df_clean)

validation_summary = pd.DataFrame(
    {
        "rows_in": [rows_in],
        "rows_out": [rows_out],
        "duplicates_removed": [duplicates_removed],
        "bad_dates_removed": [bad_dates_removed],
        "invalid_rows_removed": [invalid_rows_removed],
        "customer_missing": [customer_missing],
        "qty_nan": [qty_nan],
        "price_nan": [price_nan],
    }
)

cleaned_path = PROCESSED / "cleaned_sales.csv"
validation_path = PROCESSED / "validation_summary.csv"

df_clean.to_csv(cleaned_path, index=False)
validation_summary.to_csv(validation_path, index=False)

validation_summary

/tmp/ipython-input-3985333351.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["_parsed_date"] = pd.to_datetime(


,rows_in,rows_out,duplicates_removed,bad_dates_removed,invalid_rows_removed,customer_missing,qty_nan,price_nan
0,541929,222760,2305,311652,5212,59048,0,0


In [8]:
total_revenue = float(df_clean["Revenue"].sum())
total_orders = int(df_clean[ORDER_COL].nunique())
aov = float(total_revenue / total_orders) if total_orders else np.nan
total_quantity = float(df_clean[QTY_COL].sum())
unique_customers = (
    int(df_clean[CUSTOMER_COL].nunique())
    if CUSTOMER_COL in df_clean.columns else np.nan
)
invalid_rows_removed = float(validation_summary.loc[0, "invalid_rows_removed"])

kpi_table = pd.DataFrame(
    {
        "kpi_name": [
            "Total Revenue",
            "Total Orders",
            "Average Order Value (AOV)",
            "Total Quantity Sold",
            "Unique Customers",
            "Invalid Rows Removed",
        ],
        "kpi_value": [
            total_revenue,
            total_orders,
            aov,
            total_quantity,
            unique_customers,
            invalid_rows_removed,
        ],
        "kpi_description": [
            "Sum of Revenue across all cleaned rows.",
            "Count of unique orders/invoices after cleaning.",
            "Total Revenue divided by Total Orders.",
            "Sum of sold units (Quantity) after cleaning.",
            "Distinct customers present in cleaned data.",
            "Number of rows removed as invalid during cleaning.",
        ],
    }
)

# Revenue by Month
df_clean["year_month"] = df_clean[DATE_COL].dt.to_period("M").astype(str)
rev_by_month = (
    df_clean.groupby("year_month", as_index=False)["Revenue"]
    .sum()
    .rename(columns={"Revenue": "revenue"})
)

# Top 10 Products by Revenue
top_products = (
    df_clean.groupby(PRODUCT_COL, as_index=False)["Revenue"]
    .sum()
    .rename(columns={PRODUCT_COL: "product", "Revenue": "revenue"})
    .sort_values("revenue", ascending=False)
    .head(10)
)

# Top 10 Regions by Revenue
if REGION_COL in df_clean.columns:
    rev_by_region = (
        df_clean.groupby(REGION_COL, as_index=False)["Revenue"]
        .sum()
        .rename(columns={REGION_COL: "region", "Revenue": "revenue"})
        .sort_values("revenue", ascending=False)
        .head(10)
    )
else:
    rev_by_region = pd.DataFrame(columns=["region", "revenue"])

kpi_table.to_csv(PROCESSED / "kpi_table.csv", index=False)
rev_by_month.to_csv(PROCESSED / "rev_by_month.csv", index=False)
top_products.to_csv(PROCESSED / "top_products.csv", index=False)
rev_by_region.to_csv(PROCESSED / "rev_by_region.csv", index=False)

kpi_table

,kpi_name,kpi_value,kpi_description
0,Total Revenue,4.465471e+06,Sum of Revenue across all cleaned rows.
1,Total Orders,8.547000e+03,Count of unique orders/invoices after cleaning.
2,Average Order Value (AOV),5.224606e+02,Total Revenue divided by Total Orders.
3,Total Quantity Sold,2.281013e+06,Sum of sold units (Quantity) after cleaning.
4,Unique Customers,2.994000e+03,Distinct customers present in cleaned data.
5,Invalid Rows Removed,5.212000e+03,Number of rows removed as invalid during clean...
